In [ ]:
# Import ImageIO and PyPlot 
import imageio
import matplotlib.pyplot as plt

# Read in "chest-220.dcm"
im = imageio.imread('chest-220.dcm')
# Draw the image in grayscale
plt.imshow(im, cmap='gray')

# Render the image
plt.show()

In [ ]:
# Draw the image with greater contrast
plt.imshow(im, vmin=-200, vmax=200, cmap='gray')
plt.axis('off')
# Render the image
plt.show()

N-Dimensional Images

In [ ]:
'''In this exercise, we will use NumPy's stack() function to combine several 2D arrays into a 3D volume. By convention, volumetric data should be stacked along the first dimension: vol[plane, row, col].

Note: performing any operations on an ImageIO Image object will convert it to a numpy.ndarray, stripping its metadata.'''
# Import ImageIO and NumPy
import imageio
import numpy as np

# Read in each 2D image
im1 = imageio.imread('chest-220.dcm')
im2 = imageio.imread('chest-221.dcm')
im3 = imageio.imread('chest-222.dcm')

# Stack images into a volume
vol = np.stack([im1, im2, im3])
print('Volume dimensions:', vol.shape)

In [ ]:
'''Using volread() to load image volumes can be faster and more reliable than loading them in image-by-image. It also preserves image metadata where possible.'''
# Import ImageIO
import imageio

# Load the "tcia-chest-ct" directory
vol = imageio.volread("tcia-chest-ct")

# Print image attributes
print('Available metadata:', vol.meta.keys())
print('Shape of image array:', vol.shape)

In [ ]:
'''Field of view
The amount of physical space covered by an image is its field of view, which is calculated from two properties:
Array shape, the number of data elements on each axis. Can be accessed with the shape attribute.
Sampling resolution, the amount of physical space covered by each pixel. Sometimes available in metadata (e.g., meta['sampling']).'''
n0, n1, n2 = vol.shape
d0, d1, d2 = vol.meta['sampling']
print(n0 * d0, n1 * d1, n2 * d2)

In [ ]:
# Import PyPlot
import matplotlib.pyplot as plt
# Initialize figure and axes grid
fig, axes = plt.subplots(nrows=2, ncols=1)
# Draw an image on each subplot
axes[0].imshow(im1, cmap='gray')
axes[1].imshow(im2, cmap='gray')

# Remove ticks/labels and render
axes[0].axis('off')
axes[1].axis('off')
plt.show()

In [ ]:
'''When selecting frames, any trailing : symbols are implicitly selected. For example, vol[5] is the same as vol[5,:,:]. We will follow this simpler convention moving forward.'''
# Plot the images on a subplots array 
fig, axes = plt.subplots(nrows=1, ncols=4)

# Loop through subplots and draw image
for ii in range(4):
    im = vol[ii * 40,:,:]
    axes[ii].imshow(im, cmap='gray')
    axes[ii].axis('off')
    
# Render the figure
plt.show()

In [ ]:
# Environment Setup
from requests import get
from io import BytesIO
from zipfile import ZipFile

url1 = 'https://assets.datacamp.com/production/repositories/2085/datasets/8a007b3d94cc45c452d2c6e2bf96228b41827253/tcia-chest-ct-vol.zip'
url2 = 'https://assets.datacamp.com/production/repositories/2085/datasets/f44726fefae841afd24ddf83c58f34722212e67a/tcia-chest-ct-sample.zip'
for url in [url1, url2]:
    bfile = BytesIO(get(url).content)
    with ZipFile(bfile, "r") as z:
        z.extractall()

# Exercise Setup
import imageio
from numpy import load
import matplotlib.pyplot as plt

im = imageio.imread('chest-220.dcm')
vol = load('tcia-chest-ct.npz')['arr_0']
# vol = (vol / 255) * 4071 - 1000
vol = imageio.core.Image(vol, meta=im.meta)
vol.meta['sampling'] = (3.269999999999996, 0.976562, 0.976562)
vol.meta['shape'] = (vol.shape[0], 512, 512)
del im
# Select frame from "vol"
im1 = vol[:, 256, :]
im2 = vol[:, :, 256]

# Compute aspect ratios
d0, d1, d2 = vol.meta['sampling']
asp1 = d0 / d2
asp2 = d1 / d2

# Plot the images on a subplots array 
fig, axes = plt.subplots(nrows=2, ncols=2)

In [ ]:
# Select frame from "vol"
im1 = vol[:, 256, :]
im2 = vol[:, :, 256]

# Compute aspect ratios
d0, d1, d2 = vol.meta['sampling']
asp1 = d0 / d2
asp2 = d0 / d1

# Plot the images on a subplots array 
fig, axes = plt.subplots(nrows=2, ncols=1)
axes[0].imshow(im1, cmap='gray', aspect=asp1)
axes[1].imshow(im2, cmap='gray', aspect=asp2)

plt.show()

Mask and Filters - image intensity, mask, filter, smoothing

In [ ]:
def format_and_render_plot():
    '''Custom function to simplify common formatting operations for exercises. Operations include: 
    1. Turning off axis grids.
    2. Calling `plt.tight_layout` to improve subplot spacing.
    3. Calling `plt.show()` to render plot.'''
    fig = plt.gcf()
    fig.axes[0].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

# Load the hand radiograph
im = imageio.imread('hand-xray.jpg')
print('Data type:', np.dtype(np.uint8))
print('Min. value:', np.min(im))
print('Max value:', np.max(im))

# Plot the grayscale image
plt.imshow(im, vmin=0, vmax=255)
plt.colorbar()
format_and_render_plot()

In [ ]:
# Import SciPy's "ndimage" module
import scipy.ndimage as ndi 

# Create a histogram, binned at each possible value
hist = ndi.histogram(im, min=0, max=255, bins=256)

# Create a cumulative distribution function
cdf = hist.cumsum() / hist.sum()

# Plot the histogram and CDF
fig, axes = plt.subplots(2, 1, sharex=True)
axes[0].plot(hist, label='Histogram')
axes[1].plot(cdf, label='CDF')
format_and_render_plot()